In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import pickle
from pathlib import Path
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from statsmodels.tsa.seasonal import seasonal_decompose
from prophet import Prophet
from bs4 import BeautifulSoup
import requests
import time
from csv import writer
from datetime import datetime, timedelta
import emoji
# import streamlit_authenticator as stauth 
import schedule     

In [2]:
# Fungsi untuk melakukan scraping data
def scrape_data(start_date, end_date):
    bulan = {
        1: "Januari",
        2: "Februari",
        3: "Maret",
        4: "April",
        5: "Mei",
        6: "Juni",
        7: "Juli",
        8: "Agustus",
        9: "September",
        10: "Oktober",
        11: "November",
        12: "Desember"
    }
    data_list = []
    for date in pd.date_range(start_date, end_date):
        month_name_id = bulan[date.month]
        url_day = f"https://harga-emas.org/history-harga/{date.year}/{month_name_id}/{date.day}/"
        page = requests.get(url_day)
        soup = BeautifulSoup(page.content, 'html.parser')
        lists = soup.find('div', class_='col-md-8')
        row_data = [date.strftime('%Y-%m-%d')]
        index = 0
        for item in lists.findAll('tr'):
            index += 1
            if index == 21:
                base_value = item.findAll('b')
                index_core = 0
                for core in base_value:
                    index_core += 1
                    if index_core == 2:
                        value = core.text.split('+')[0].split('-')[0].split('(')[0]
                        value = value.replace('.', '').strip()
                        value = value.replace('Rp', '').strip()
                        value = value.replace('/', '').strip()
                        value = value.replace('gram', '').strip()
                        row_data.append(value)
        data_list.append(row_data)
        time.sleep(1)  # Jeda untuk mencegah terlalu banyak permintaan ke website
    return data_list

In [3]:
def save_to_csv(data_list, file_name):
    with open(file_name, 'a', newline='') as file:
        csv_writer = writer(file)
        csv_writer.writerows(data_list)

In [4]:
def scrape_dan_simpan():
    # Baca tanggal terakhir dari file CSV
    file_name = 'data/harga_emas.csv'
    df5 = pd.read_csv(file_name)
    last_date_str = df5.iloc[-1]['Tanggal']
    last_date = datetime.strptime(last_date_str, '%Y-%m-%d').date()
        
    start_date = last_date + timedelta(days=1)
    end_date = datetime.now().date()
        
    if start_date <= end_date:
        # Scraping data dan menyimpannya ke dalam file CSV
        data_list = scrape_data(start_date, end_date)
        save_to_csv(data_list, file_name)

In [5]:
# Load data
df = pd.read_csv('data/harga_emas.csv')
df['Tanggal'] = pd.to_datetime(df['Tanggal'])
df['Tanggal'] = df['Tanggal'].dt.date
df.set_index('Tanggal', inplace=True)
df['Harga'] = (df['Price1'])
print(df.dtypes)
df.head(5)

Price100    int64
Price50     int64
Price25     int64
Price10     int64
Price5      int64
Price3      int64
Price2      int64
Price1      int64
Harga       int64
dtype: object


,Price100,Price50,Price25,Price10,Price5,Price3,Price2,Price1,Harga
Tanggal,,,,,,,,,
2014-01-01,48550000,24300000,12175000,4900000,2475000,1494000,1008000,524000,524000
2014-01-02,49150000,24600000,12325000,4960000,2505000,1512000,1020000,530000,530000
2014-01-03,49350000,24700000,12375000,4980000,2515000,1518000,1024000,532000,532000
2014-01-04,49350000,24700000,12375000,4980000,2515000,1518000,1024000,532000,532000
2014-01-05,49350000,24700000,12375000,4980000,2515000,1518000,1024000,532000,532000


In [6]:
# df.dropna(inplace = True)
# df.reset_index(inplace = True)
df.drop(['Price1', 'Price2', 'Price3', 'Price5', 'Price10', 'Price25', 'Price50', 'Price100'], axis=1, inplace=True)
df.head(5)

,Harga
Tanggal,
2014-01-01,524000
2014-01-02,530000
2014-01-03,532000
2014-01-04,532000
2014-01-05,532000


In [7]:
# #load data lagi
# df2 = pd.read_csv('data/harga_emas.csv')
# df2['Tanggal'] = pd.to_datetime(df2['Tanggal'])
# df2.set_index('Tanggal', inplace=True)
# print(df2.dtypes)
# df2.head(5)

In [8]:
# df.dropna(inplace = True)
# df.reset_index(inplace = True)
# df2.drop(['Price1', 'Price2', 'Price3', 'Price5', 'Price10', 'Price25', 'Price50', 'Price100'], axis=1, inplace=True)
# df.head(5)

In [9]:
data_set = df.iloc[:, 0:1] #.values
pd.set_option('display.max_columns', None)

data_set.head(5)

,Harga
Tanggal,
2014-01-01,524000
2014-01-02,530000
2014-01-03,532000
2014-01-04,532000
2014-01-05,532000


In [10]:
# Learning / Preprocessing data
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
data_set_scaled = sc.fit_transform(data_set)
print(data_set_scaled)

[[0.0094451 ]
 [0.01652893]
 [0.0188902 ]
 ...
 [0.95513577]
 [0.95749705]
 [0.95749705]]


In [11]:
# Multiple features from data provided to the model
X = []
backcandles = 30 # Jumlah hari mundur / kebelakang
print(data_set_scaled.shape[0])
for j in range(1): # jumlah kolom = 8
  X.append([])
  for i in range(backcandles, data_set_scaled.shape[0]):
    X[j].append(data_set_scaled[i - backcandles:i, j])

X = np.moveaxis(X, [0], [2])

# -1 untuk memilih kolom terakhir
X, yi = np.array(X), np.array(data_set_scaled[backcandles:, -1])
y=np.reshape(yi,(len(yi),1))
# y = yi.reshape(-1, 1)

print(X.shape)
print(y)
print(y.shape)

4681
(4651, 30, 1)
[[0.02715466]
 [0.02715466]
 [0.02715466]
 ...
 [0.95513577]
 [0.95749705]
 [0.95749705]]
(4651, 1)


In [12]:
# Split data into train test sets
splitlimit = int(len(X)*0.8)
print(splitlimit)

X_train, X_test = X[:splitlimit], X[splitlimit:]
y_train, y_test = y[:splitlimit], y[splitlimit:]
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(y.shape)

3720
(3720, 30, 1)
(931, 30, 1)
(3720, 1)
(931, 1)
(4651, 1)


In [13]:
# from keras.models import Sequential

# import tensorflow as tf
# import keras
# from keras import optimizers
# from keras.callbacks import History
# from keras.models import Model
# from keras.layers import Dense, Dropout, LSTM, Input, Activation, concatenate, TimeDistributed
# import numpy as np
# import matplotlib
# matplotlib.use('WebAgg')
# # from matplotlib.figure import Figure
# import matplotlib.pyplot as plt

# # backcandles = 30

# lstm_input = Input(shape=(backcandles, 1), name="lstm_input")
# print("Backcandles: ", + backcandles)
# inputs = LSTM(150, name="first_layer")(lstm_input)
# inputs = Dense(1, name="dense_layer")(inputs)
# inputs = Dropout(0.2)(inputs)
# output = Activation('linear', name="output")(inputs)
# model = Model(inputs=lstm_input, outputs=output)
# adam = optimizers.Adam(learning_rate=0.001)
# model.compile(optimizer=adam, loss='mse')

# print("X_train shape:", X_train.shape)
# print("X_train dtype:", X_train.dtype)
# print("y_train shape:", y_train.shape)
# print("y_train dtype:", y_train.dtype)

# model_test = model.fit(x=X_train, y=y_train, batch_size=15, epochs=30, shuffle=True, validation_split=0.1)

# plt.close('all')
# plt.plot(model_test.history['loss'])
# plt.plot(model_test.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'validation'], loc='upper right')
# plt.show()
# # plt.close()

In [14]:
# MANUAL HYPERPARAMETERS TUNING

import tensorflow as tf
from keras.models import Model
from keras.layers import Dense, Dropout, LSTM, Input, Activation
from keras import optimizers
import numpy as np
import matplotlib
matplotlib.use('WebAgg')
import matplotlib.pyplot as plt

# Hyperparameters to tune
backcandles = 30  # Assuming 30 time steps
lstm_units = 150  # Start with 150, tune this value
learning_rate = 0.001  # Start with 0.001, tune this value
batch_size = 30  # Start with 15, tune this value
epochs = 30  # Start with 30, tune this value
dropout_rate = 0.2  # Start with 0.2, tune this value

lstm_input = Input(shape=(backcandles, 1), name="lstm_input")
inputs = LSTM(lstm_units, name="first_layer")(lstm_input)
inputs = Dropout(dropout_rate)(inputs)  # Added dropout layer
inputs = Dense(1, name="dense_layer")(inputs)
output = Activation('linear', name="output")(inputs)
model = Model(inputs=lstm_input, outputs=output)

adam = optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=adam, loss='mse')

print("X_train shape:", X_train.shape)
print("X_train dtype:", X_train.dtype)
print("y_train shape:", y_train.shape)
print("y_train dtype:", y_train.dtype)

model_test = model.fit(x=X_train, y=y_train, batch_size=batch_size, epochs=epochs, shuffle=True, validation_split=0.1)

plt.close('all')
plt.plot(model_test.history['loss'])
plt.plot(model_test.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()


X_train shape: (3720, 30, 1)
X_train dtype: float64
y_train shape: (3720, 1)
y_train dtype: float64
Epoch 1/30
112/112 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 0.0114 - val_loss: 0.0013
Epoch 2/30
112/112 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 0.0010 - val_loss: 0.0014
Epoch 3/30
112/112 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 7.6633e-04 - val_loss: 0.0019
Epoch 4/30
112/112 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 8.0771e-04 - val_loss: 9.4746e-04
Epoch 5/30
112/112 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 6.9516e-04 - val_loss: 0.0023
Epoch 6/30
112/112 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 8.2102e-04 - val_loss: 8.1071e-04
Epoch 7/30
112/112 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 6.5920e-04 - val_loss: 6.7169e-04
Epoch 8/30
112/112 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 6.2042e-04 - val_loss: 0.0016
Epoch 9/30
112/112 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 6.5925e-04 - val_loss: 6.3007e-04
Epoch 10/30
112/112 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 5.3198e-04 

In [15]:
# # AUTOMATIC HYPERPARAMETERS TUNING

# # from gc import callbacks
# import tensorflow as tf
# # from keras import Sequential, callbacks
# # from keras.layers import Dense, Dropout, LSTM, Input, Activation
# # from keras import optimizers

# # def build_model(hp):
# #     model = Sequential()
# #     model.add(LSTM(units=hp.Int('units', min_value=50, max_value=300, step=50), input_shape=(backcandles, 1)))
# #     model.add(Dropout(rate=hp.Float('dropout', min_value=0.1, max_value=0.5, step=0.1)))
# #     model.add(Dense(1))
# #     model.add(Activation('linear'))

# #     model.compile(optimizer=optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[0.001, 0.0001])),
# #                   loss='mse')
# #     return model

# def model_builder(hp):
#   model = tf.keras.Sequential()
#   model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))

#   hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
#   hp_layer_1 = hp.Int('layer_1', min_value=1, max_value=1000, step=100)
#   hp_layer_2 = hp.Int('layer_2', min_value=1, max_value=1000, step=100)
#   hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

#   model.add(tf.keras.layers.Dense(units=hp_layer_1, activation=hp_activation))
#   model.add(tf.keras.layers.Dense(units=hp_layer_2, activation=hp_activation))
#   model.add(tf.keras.layers.Dense(10, activation='softmax'))

#   model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
#                 loss=tf.keras.losses.SparseCategoricalCrossentropy(),
#                 metrics=['accuracy'])
  
#   return model


# # tuner = kt.Hyperband(build_model,
# #                      objective='val_loss',
# #                      max_epochs=15,
# #                      factor=3,
# #                      directory='my_dir',
# #                      project_name='intro_to_kt')

# import keras_tuner as kt

# tuner = kt.Hyperband(model_builder,
#                      objective='val_accuracy',
#                      max_epochs=10,
#                      factor=3,
#                      directory='dir',
#                      project_name='x')

# # stop_early = callbacks.EarlyStopping(monitor="val_loss", patience = 3)
# stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
# tuner.search(x=X_train, y=y_train, epochs=30, validation_split=0.2, callbacks=[stop_early])


# # Get the optimal hyperparameters
# best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# print(f"""
# The optimal number of units in the first LSTM layer is {best_hps.get('units')}.
# The optimal dropout rate is {best_hps.get('dropout')}.
# The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.
# """)


In [16]:
# from sklearn import metrics
# from sqlalchemy import values
# import tensorflow as tf
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Dense, Dropout, LSTM, Input, Activation
# from tensorflow.keras import optimizers
# import numpy as np
# import matplotlib
# matplotlib.use('WebAgg')
# import matplotlib.pyplot as plt
# import keras_tuner as kt

# # Example data (replace with your actual data)
# # X_train = np.random.rand(3740, 30, 1)
# # y_train = np.random.rand(3740, 1)

# # Hyperparameter tuning function
# def build_model(hp):
#     lstm_units = hp.Int('units', min_value=50, max_value=300, step=50)
#     batch_size = hp.Int('batch_size', min_value=10, max_value=1000, step=10)
#     dropout_rate = hp.Float('dropout', min_value=0.1, max_value=0.5, step=0.1)
#     learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
#     activation = hp.Choice('activation', values=['relu', 'tanh'])
    
#     lstm_input = Input(shape=(30, 1), name="lstm_input")
#     inputs = LSTM(lstm_units, name="first_layer")(lstm_input)
#     inputs = Dropout(dropout_rate)(inputs)
#     inputs = Dense(1, name="dense_layer")(inputs)
#     # inputs = Activation(activation)
#     # inputs = batch_size
#     output = Activation('linear', name="output")(inputs)
#     model = Model(inputs=lstm_input, outputs=output)
    
#     adam = optimizers.Adam(learning_rate=learning_rate)
#     # model.compile(optimizer=adam, loss='mse')
#     model.compile(optimizer=adam, loss='mse', metrics=['accuracy'])
    
#     return model

# # Set up the tuner
# # tuner = kt.RandomSearch(
# tuner = kt.Hyperband(
#     build_model,
#     objective='val_loss',
#     max_epochs=30,
#     factor=3,
#     executions_per_trial=1,
#     directory='my_dir',
#     project_name='lstm_tuning',
#     overwrite=True,
# )

# # def model_builder(hp):
# #   model = tf.keras.Sequential()
# #   # model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))

# #   hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
# #   hp_layer_1 = hp.Int('layer_1', min_value=1, max_value=1000, step=100)
# #   hp_layer_2 = hp.Int('layer_2', min_value=1, max_value=1000, step=100)
# #   hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

# #   model.add(tf.keras.layers.Dense(units=hp_layer_1, activation=hp_activation))
# #   model.add(tf.keras.layers.Dense(units=hp_layer_2, activation=hp_activation))
# #   model.add(tf.keras.layers.Dense(10, activation='softmax'))

# #   model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
# #                 loss=tf.keras.losses.SparseCategoricalCrossentropy(),
# #                 metrics=['accuracy'])
  
# #   return model

# # import keras_tuner as kt

# # tuner = kt.Hyperband(model_builder,
# #                      objective='val_accuracy',
# #                      max_epochs=10,
# #                      factor=3,
# #                      directory='dir',
# #                      project_name='x')

# # Display search space summary
# tuner.search_space_summary()

# # Define the EarlyStopping callback
# stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

# # Perform the hyperparameter search
# # tuner.search(x=X_train, y=y_train, epochs=30, validation_split=0.2, callbacks=[stop_early])
# tuner.search(
#     X_train,
#     y_train,
#     epochs=30,
#     validation_split=0.2,
#     callbacks=[stop_early],
#     # batch_size=kt.In('batch_size', min_value=10, max_value=1000, step=10)
# )

# # Retrieve the best model and hyperparameters
# best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# print(f"""
# The optimal number of units in the first LSTM layer is {best_hps.get('units')}.
# The optimal dropout rate is {best_hps.get('dropout')}.
# The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.
# """)

# # Build the best model
# model = tuner.hypermodel.build(best_hps)

# # Getting the most optimal batch_size
# optimal_batch_size = best_hps.get('batch_size')

# # Train the model
# history = model.fit(
#     X_train,
#     y_train,
#     epochs=30,
#     validation_split=0.1,
#     # batch_size=best_hps.get('batch_size'),
#     batch_size=optimal_batch_size,
#     callbacks=[stop_early]
# )

# # Plot the results
# plt.close('all')
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('Model Loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Validation'], loc='upper right')
# plt.show()


In [17]:
# # Build the best model
# model = tuner.hypermodel.build(best_hps)
# history = model.fit(X_train, y_train, epochs=30, validation_split=0.1, callbacks=['stop_early'])

In [18]:
# import tensorflow as tf
# from keras.models import Model
# from keras.layers import Dense, Dropout, LSTM, Input, Activation
# from keras import optimizers
# import numpy as np
# import matplotlib
# matplotlib.use('WebAgg')
# import matplotlib.pyplot as plt
# from keras_tuner import RandomSearch

# # Example data (replace with your actual data)
# # X_train = np.random.rand(3740, 30, 1)
# # y_train = np.random.rand(3740, 1)

# # Hyperparameter tuning function
# def build_model(hp):
#     lstm_units = hp.Int('units', min_value=50, max_value=300, step=50)
#     dropout_rate = hp.Float('dropout', min_value=0.1, max_value=0.5, step=0.1)
#     learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
#     lstm_input = Input(shape=(30, 1), name="lstm_input")
#     inputs = LSTM(lstm_units, name="first_layer")(lstm_input)
#     inputs = Dropout(dropout_rate)(inputs)
#     inputs = Dense(1, name="dense_layer")(inputs)
#     output = Activation('linear', name="output")(inputs)
#     model = Model(inputs=lstm_input, outputs=output)
    
#     adam = optimizers.Adam(learning_rate=learning_rate)
#     model.compile(optimizer=adam, loss='mse')
    
#     return model

# # Set up the tuner
# tuner = RandomSearch(
#     build_model,
#     objective='val_loss',
#     max_trials=10,
#     executions_per_trial=1,
#     directory='my_dir',
#     project_name='lstm_tuning'
# )

# # Display search space summary
# tuner.search_space_summary()

# # Perform the hyperparameter search
# tuner.search(X_train, y_train, epochs=30, validation_split=0.1)

# # Retrieve the best model and hyperparameters
# best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# print(f"""
# The optimal number of units in the first LSTM layer is {best_hps.get('units')}.
# The optimal dropout rate is {best_hps.get('dropout')}.
# The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.
# """)

# # Build the best model
# model = tuner.hypermodel.build(best_hps)
# history = model.fit(X_train, y_train, epochs=30, validation_split=0.1)

# # Plot the training history
# plt.close('all')
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('Model Loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Validation'], loc='upper right')
# plt.show()


In [19]:
# Prediction
y_pred = model.predict(X_test)
for i in range(10):
  print(y_pred[i], y_test[i])
  
# plt.figure(figsize=(8,4))
# plt.plot(y_test, color="red", label="Test")
# plt.plot(y_pred, color="blue", label="Prediction")
# plt.legend()
# plt.show()

30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
[0.63471645] [0.63990555]
[0.63772094] [0.63400236]
[0.63712704] [0.63046045]
[0.6346474] [0.64108619]
[0.63845384] [0.64108619]
[0.6413568] [0.64108619]
[0.6430212] [0.64698937]
[0.6466515] [0.65525384]
[0.6529305] [0.66824085]
[0.663342] [0.66587957]


In [20]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# MSE
mse = mean_squared_error(y_test, y_pred)
print("MSE score: ", mse)

# RMSE
rmse = np.sqrt(mse)
print("RMSE score: ", rmse)

# MAE
mae = mean_absolute_error(y_test, y_pred)
print("MAE score: ", mae)

#R2
r2 = r2_score(y_test, y_pred)
print("r2_score: ", r2)

def mean_absolute_percentage_error(y_test, y_pred):
    return np.mean(np.abs((y_test - y_pred) / y_pred)) * 100

mape = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE: ", mape, "%")

MSE score:  0.00047174997313834404
RMSE score:  0.021719806010605712
MAE score:  0.011311807733772177
r2_score:  0.9541855553677676
MAPE:  1.5743837143437431 %


In [21]:
plt.figure(figsize=(12,6))
plt.plot(y_test, color="red", label="Test")
plt.plot(y_pred, color="blue", label="Prediction")
plt.legend()
plt.show()
# plt.close()

In [22]:
# Get the last backcandles days' price
last_backandles_prices = data_set_scaled[-backcandles:]

# Reshape the data to match the modul input shape
last_backandles_prices = last_backandles_prices.reshape((1, backcandles, 1))

# Predict tomorrow's price
predicted_tomorrow_price = model.predict(last_backandles_prices)

# Inverse transform the predicted price to get the actual price
predicted_tomorrow_price = sc.inverse_transform(predicted_tomorrow_price)

print("Tomorrow's price: ", predicted_tomorrow_price[0][0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Tomorrow's price:  1316950.6


Task exception was never retrieved
future: <Task finished name='Task-162' coro=<WebSocketProtocol13.write_message.<locals>.wrapper() done, defined at c:\Users\LENOVO\anaconda3\Lib\site-packages\tornado\websocket.py:1085> exception=WebSocketClosedError()>
Traceback (most recent call last):
  File "c:\Users\LENOVO\anaconda3\Lib\site-packages\tornado\websocket.py", line 1087, in wrapper
    await fut
tornado.iostream.StreamClosedError: Stream is closed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\LENOVO\anaconda3\Lib\site-packages\tornado\websocket.py", line 1089, in wrapper
    raise WebSocketClosedError()
tornado.websocket.WebSocketClosedError
Task exception was never retrieved
future: <Task finished name='Task-165' coro=<WebSocketProtocol13.write_message.<locals>.wrapper() done, defined at c:\Users\LENOVO\anaconda3\Lib\site-packages\tornado\websocket.py:1085> exception=WebSocketClosedError()>
Traceback (most r